Hall Effect

https://en.wikipedia.org/wiki/Hall_effect#Corbino_effect

The Hall effect is the production of a potential difference (the Hall voltage) across an electrical conductor that is transverse to an electric current in the conductor and to an applied magnetic field perpendicular to the current.

U_H = h L I B / (N q_e)

https://www.youtube.com/watch?v=1OZCWetFCps

In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi

functions for generating monomials

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

In [3]:
@polyvar F_m q_e v B F_e E h U_H dt L I Q N n V d   # the order matters per evaluation(?) # a_y a_x dv_x dv_y

x =     [F_m q_e v B F_e E h U_H dt L I Q N n V d  ] #dv_x dv_y a_y a_x

1×16 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 F_m  q_e  v  B  F_e  E  h  U_H  dt  L  I  Q  N  n  V  d

In [4]:
axioms= 
[
    F_m - q_e * v * B,                    # Lorentz force
    F_e - q_e * E,                        # electric force (E homogenous electric feild between the upper and lower metal plate)
    F_m - F_e,                            # Newton's 3rd Law
    E * h - U_H,                          # electric potnetial U_H to field E relation per plate width h (the electric field is not known)
    v * dt - L,                           # velocity (v = L/dt) is not known
    I * dt - Q,                           # amount of charge Q, is the current I times dt (can be measured by an amp meter)
    Q - N * q_e,                          # the charge Q is the number of electrons N by the chage of each electron q_e
    n * V - N,                            # charge density is the number of electrons traveling through the metal plate in a given time over the volume V
    V - L * h * d,                        # the volume V is the product of the length L by the width h, by the thickness of the plate d
    ]
    

9-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 F_m - q_evB
 F_e - q_eE
 -F_e + F_m
 -U_H + Eh
 -L + vdt
 -Q + dtI
 Q - q_eN
 -N + nV
 V - hLd

In [5]:
deg = 3
deg_overall = 8
theDegrees = [ 1   1  2 1  1  1 2  1  2  2 2 1 1 1 1 1] 
            # F_m q_e v B F_e E h U_H dt L I Q N n V d

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([I n q_e d B h L U_H], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [U_H ]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(104583,), (104583,), (104583,), (104583,), (104583,), (104583,), (104583,), (104583,), (104583,)]


In [14]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#75 (generic function with 1 method)

In [15]:
value_q = value_poly(q)
value_q

2.0mGM + mv_e²r

In [3]:
@show value_αs = value_poly.(αs)

LoadError: UndefVarError: `value_poly` not defined

___________________________________________________________________reduced parameters_____________________________________________________________

In [7]:
@polyvar F_m q_e v B F_e E h U_H dt L I N   # the order matters per evaluation # Q N n V d

x =     [F_m q_e v B F_e E h U_H dt L I N  ] #dv_x dv_y a_y a_x

1×12 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 F_m  q_e  v  B  F_e  E  h  U_H  dt  L  I  N

In [8]:
axioms= 
[
    F_m - q_e * v * B,                    # Lorentz force
    F_e - q_e * E,                        # electric force (E homogenous electric feild between the upper and lower metal plate)
    F_m - F_e,                            # Newton's 3rd Law
    E * h - U_H,                          # electric potnetial U_H to field E relation per plate width h (the electric field is not known)
    v * dt - L,                           # velocity (v = L/dt) is not known
  #  I * dt - Q,                           # amount of charge Q, is the current I times dt (can be measured by an amp meter)
    I * dt - N * q_e,                       # amount of charge Q, is the current I times dt (can be measured by an amp meter)
   # Q - N * q_e,                          # the charge Q is the number of electrons N by the chage of each electron q_e
   ## n * L * h * d - N,                      # charge density is the number of electrons traveling through the metal plate in a given time over the volume V
   # n * V - N,                            # charge density is the number of electrons traveling through the metal plate in a given time over the volume V
   # V - L * h * d,                        # the volume V is the product of the length L by the width h, by the thickness of the plate d
    ]
    

6-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 F_m - q_evB
 F_e - q_eE
 -F_e + F_m
 -U_H + Eh
 -L + vdt
 dtI - q_eN

In [11]:
deg = 2
deg_overall = 8
theDegrees = [ 1   2  2 2  1  2 2  2  1  1 1 1] 
            # F_m q_e v B F_e E h U_H dt L I N  

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([I N q_e B h L U_H], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [U_H ]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(19438,), (19438,), (19438,), (19438,), (19438,), (19438,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
Set parameter TimeLimit to value 600
size.(coeff_αs) = [(19438,), (19438,), (19438,), (19438,), (19438,), (19438,)]
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 114894 rows, 118275 columns and 235899 nonzeros
Model fingerprint: 0x8063a4d5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 114894 rows and 118275 columns
Presolve time: 0.10s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf. 

OPTIMAL::TerminationStatusCode = 1

In [12]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#45 (generic function with 1 method)

In [13]:
value_q = value_poly(q)
value_q

q_e²U_HLN² - q_eBhL²IN

After rearangment we get  U_H  = B h L I / (N q_e)

In [14]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[hdtLIN, -hdtLIN, -hdtLIN, -q_edtLIN, q_eBhLIN, -q_eU_HLN]


6-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 hdtLIN
 -hdtLIN
 -hdtLIN
 -q_edtLIN
 q_eBhLIN
 -q_eU_HLN

______________________________________________________________ eliminate q ____________________________________________________________________________________

In [15]:
@polyvar F_m q_e v B F_e E h U_H dt L I N n Q d # the order matters per evaluation # Q N n V d

x =     [F_m q_e v B F_e E h U_H dt L I N n Q d] #dv_x dv_y a_y a_x

1×15 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 F_m  q_e  v  B  F_e  E  h  U_H  dt  L  I  N  n  Q  d

In [16]:
axioms= 
[
    F_m - q_e * v * B,                    # Lorentz force
    F_e - q_e * E,                        # electric force (E homogenous electric feild between the upper and lower metal plate)
    F_m - F_e,                            # Newton's 3rd Law
    E * h - U_H,                          # electric potnetial U_H to field E relation per plate width h (the electric field is not known)
    v * dt - L,                           # velocity (v = L/dt) is not known
  #  I * dt - Q,                           # amount of charge Q, is the current I times dt (can be measured by an amp meter)
    I * dt - N * q_e,                       # amount of charge Q, is the current I times dt (can be measured by an amp meter)
   # Q - N * q_e,                          # the charge Q is the number of electrons N by the chage of each electron q_e
    n * L * h * d - N,                      # charge density is the number of electrons traveling through the metal plate in a given time over the volume V
   # n * V - N,                            # charge density is the number of electrons traveling through the metal plate in a given time over the volume V
   # V - L * h * d,                        # the volume V is the product of the length L by the width h, by the thickness of the plate d
    ]

7-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 F_m - q_evB
 F_e - q_eE
 -F_e + F_m
 -U_H + Eh
 -L + vdt
 dtI - q_eN
 -N + hLnd

In [17]:
deg = 2
deg_overall = 8
theDegrees = [ 1   2  2 2  1  2 2  2  1  1 1 1 1 1 1] 
            # F_m q_e v B F_e E h U_H dt L I N n Q d  

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([I n q_e B h d U_H], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [U_H ]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(79665,), (79665,), (79665,), (79665,), (79665,), (79665,), (79665,)]
